In [1]:
# load packages
import os
import glob

import time
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd
from tkinter import filedialog
from tkinter import *

import matplotlib.pyplot as plt
import matplotlib.image as plt_img
from matplotlib.offsetbox import AnchoredText

import h5py
import cv2
#from guppy import hpy
import gc
import shlex
import pipes
from subprocess import check_call
import fnv
import fnv.reduce
import fnv.file  
from tqdm import tqdm
from joblib import Parallel, delayed
 
gc.collect()
#h5py.run_tests()

0

In [2]:
def plot_thermal_frame(i: int, it: int, im, fig):
    """
    Plot a thermal frame with a timestamp and save it as a TIFF file.

    Args:
        i (int): Index of the frame.
        it (Iterator): An iterator object.
        im (Image): An Image object.
        fig (Figure): A Figure object.
    Returns:
        None
    """
#    fig = plt.figure(figsize=(width, height), dpi=100)
    
    # Get the current frame and reshape it to the height and width of the image.
    im.get_frame(it)
    final = np.array(im.final, copy=False).reshape((im.height, im.width))
    final1 = final - final.mean()

    plt.imshow(final1, vmin=-np.quantile(final1, .5), vmax=final1.max(), cmap='rocket', interpolation='catrom')

    timestamp = im.frame_info[0]['value'][4:]
    timestamp = dt.datetime.strptime(timestamp, '%H:%M:%S.%f')
    text_box = AnchoredText(timestamp.strftime("%H:%M:%S.%f")[:-5] + "_" + str(it), frameon=True, loc=2, pad=0.5,
                            prop=dict(fontsize=12))
    plt.setp(text_box.patch, facecolor='white', alpha=0.2)
    plt.gca().add_artist(text_box)

    # Set axis and save the plot as a TIFF file.
    plt.gca().set_position([0, 0, 1, 1])
    plt.axis('off')
    plt.savefig(f'{i:06}.tiff', bbox_inches='tight', pad_inches=0)

    # Close the plot to save memory
    plt.close()


In [3]:
## setting parameters and folders
zoom = 1
vid_length = 1*60*60

main_folder = '//mnt/EAS_shared/baboon/working/data/'
data_folder =  main_folder + 'raw/2019/thermal/cliff_data/viewpoint_1/T1020/'

image_folder = main_folder + 'processed/2019/thermal/cliff_data/T1020/frames_highres/'
output_folder =  main_folder + 'processed/2019/thermal/cliff_data/T1020/full_nights/'

if not os.path.isdir( image_folder ):
    os.makedirs ( image_folder)

if not os.path.isdir( output_folder ):
    os.makedirs ( output_folder)
    
os.chdir(data_folder)
file_names = glob.glob('**/*.seq', recursive=True)



In [4]:
for file_name in file_names:
    print(file_name)
    im = fnv.file.ImagerFile(data_folder + file_name)

    im.get_frame(0)
    #im.frame_info[1]
    #specifying first and last frame after loading the specific file
    #print('got_frame')

    # selected frames to make the video
    start_frame = 0
    end_frame = im.num_frames
    step_frame = 25
    frame_range = range(start_frame,end_frame,step_frame)
    print(frame_range)

    vid_name = file_name[:8] + '_' + str(end_frame) + '_' + str(step_frame)
    if not os.path.exists(output_folder + vid_name + '.mp4'):
        # where to write the images to   
        #day1 = int(im.frame_info[0]['value'][:3])
        #start = f'{start_dt.year:04}-' + f'{start_dt.month:02}-' + f'{start_dt.day:02} ' + im.frame_info[0]['value'][4:]
        #start_dt = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f')

        os.chdir( image_folder )
        for f in glob.glob(image_folder + "*.tiff"):
            os.remove(f)

        final = np.array(im.final, copy=False).reshape((im.height, im.width))
        width = final.shape[1] / (100 / zoom)  
        height = final.shape[0] / (100 / zoom)

        fig = plt.figure( figsize = ( width, height ) )  
        fig.set_dpi(100)

        # Loop through frame_range and make frames with plot_thermal_frame function
        for idx, item in tqdm(enumerate(frame_range)):
            plot_thermal_frame(idx, item)

        del im
        gc.collect()

        os.chdir(main_folder)
        command = 'ffmpeg -i ' + image_folder + '%06d.tiff -c:v libx264 -qp 0 '  + output_folder + vid_name + '.mp4'
        check_call(shlex.split(command))
        

20190810/20190810_16_30_00-222_13_30_02_972.seq
range(0, 35044, 5)
20190803/20190803_16_30_00-215_13_30_06_187.seq
range(0, 222758, 5)
20190809/20190809_16_30_00-221_13_30_08_080.seq
range(0, 90098, 5)
20190804/20190804_16_30_00-216_13_30_09_112.seq
range(0, 209510, 5)
20190817/20190817_17_00_00-229_14_00_02_320.seq
range(0, 95527, 5)
20190821/20190821_16_01_00-233_13_01_00_924.seq
range(0, 165911, 5)
20190813/20190813_16_30_00-225_13_30_05_916.seq
range(0, 57572, 5)
20190819/20190819_16_20_00-231_13_20_01_056.seq
range(0, 77092, 5)
20190807/20190807_17_00_00-219_14_00_05_282.seq
range(0, 250370, 5)
20190814/20190814_17_00_00-226_14_00_04_981.seq
range(0, 110725, 5)
20190816/20190816_16_30_00-228_13_30_06_090.seq
range(0, 139240, 5)
20190808/20190808_16_30_00-220_13_30_05_390.seq
range(0, 191451, 5)
20190802/20190802_16_05_00-000008-214_13_05_07_052.seq
range(0, 93601, 5)
20190811/20190811_16_30_00-223_13_30_05_954.seq
range(0, 143755, 5)
20190815/20190815_16_35_00-227_13_35_05_016.seq

In [ ]:
# choose file
file_name = file_names[4]
print(file_name)

# load file and basic plot
im = fnv.file.ImagerFile(data_folder + file_name)
im.get_frame(0)
template_frame = np.array(im.final, copy=False).reshape((im.height, im.width))
width = template_frame.shape[1] / (100 / zoom)  
height = template_frame.shape[0] / (100 / zoom)

fig = plt.figure( figsize = ( width, height ) )  
fig.set_dpi(100)

vid_name = file_name[:8] + '_' + str(end_frame) + '_' + str(step_frame)

In [ ]:
## writing all frames from a given thermal recording, separated into minute bins. Timestamps unrounded
if not os.path.exists(output_folder + vid_name + '.mp4'):
    os.chdir( image_folder )
    for f in glob.glob(image_folder + "*.tiff"):
        os.remove(f)

    final = np.array(im.final, copy=False).reshape((im.height, im.width))
    width = final.shape[1] / (100 / zoom)  
    height = final.shape[0] / (100 / zoom)

    fig = plt.figure( figsize = ( width, height ) )  
    fig.set_dpi(100)

    # Loop through frame_range and make frames with plot_thermal_frame function
    for idx, item in tqdm(enumerate(frame_range)):
        plot_thermal_frame(idx, item, im, fig)

    del im
    gc.collect()

    os.chdir(main_folder)
    command = 'ffmpeg -i ' + image_folder + '%06d.tiff -c:v libx264 -qp 0 '  + output_folder + vid_name + '.mp4'
    check_call(shlex.split(command))